In [0]:
%%sh
PGPASSWORD='dNzWtSV3pKTx' psql -h 10.2.10.132 -Ucitus_bdp_usage_qa -d aa_citus_db -p 5432 << EOF 

SET search_path=advancedreview;

EXPLAIN ANALYZE select max(term) as term,
ROUND((pow(sum(edf), 2) * (AVG(rating) - 4.5) * AVG(LN(score)) / count(*))::numeric, 1) as fact_review_phrase_impact_score__aggr,
count(*) OVER() AS total
from (
    select advancedreview_term_fact_v1.review_id as review_id,
    unnest(advancedreview_term_fact_v1.terms) as term,
    unnest(advancedreview_term_fact_v1.term_ids) as term_id,
    unnest(advancedreview_term_fact_v1.scores) as score,
    unnest(advancedreview_term_fact_v1.is_extracted) as edf,
    advancedreview_term_fact_v1.rating as rating
    from advancedreview_term_fact_v1
    INNER JOIN advancedreview_topic_fact_v1
    ON advancedreview_topic_fact_v1.review_id = advancedreview_term_fact_v1.review_id
    and advancedreview_topic_fact_v1.product_id = advancedreview_term_fact_v1.product_id
    where
        advancedreview_term_fact_v1.product_id = 284882215
    AND advancedreview_term_fact_v1.country_code = 'US'
    AND advancedreview_topic_fact_v1.product_id = 284882215
    AND advancedreview_topic_fact_v1.country_code = 'US'
    and advancedreview_topic_fact_v1.topic_ids @> ARRAY[10]::smallint[]) as sub_table
GROUP BY term_id

EOF 


In [0]:
%%sh
